<a href="https://colab.research.google.com/github/0xLighty/Natural-Language-Processing/blob/main/hw4_task1_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1
Que 1: Fine tune the BERT for question answering model on coqa dataset using similar process as shown in the demo for the squad dataset

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.7 MB/s 
     |████████████████████████████████| 7.6 MB 47.7 MB/s 
     |████████████████████████████████| 182 kB 59.2 MB/s 


In [2]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# importing required libraries
import requests
import json
import torch
import os
from tqdm import tqdm
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering
from transformers import AdamW

import pandas as pd
import numpy as np
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [4]:
# loading the dataset
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...,...
7194,1,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,1,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,1,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,1,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [5]:
# deleting the unrequired columns
del coqa["version"]
coqa

,data
0,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...
7194,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [6]:
coqa

,data
0,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...
7194,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [20]:
# structuring the dataset in usable format (creating one question and answer pair per row)
# by doing this, the text column will have repeated content until all questions of that paragraph are not covered
# structure: 
# cols = ["text","question","answer"]
# comp_list = []
# end_list = []
# for index, row in coqa.iterrows():
#     for i in range(len(row["data"]["questions"])):
#         if row["data"]["answers"][i]["input_text"] == 'unknown':
#           continue
#         temp_list = []
#         temp_list.append(row["data"]["story"])
#         temp_list.append(row["data"]["questions"][i]["input_text"])
#         temp_list.append({'text': row["data"]["answers"][i]["input_text"], 'answer_start': row["data"]["answers"][i]["span_start"]})
#         end_list.append(row["data"]["answers"][i]["span_end"])
#         comp_list.append(temp_list)
# train_df = pd.DataFrame(comp_list, columns = cols) 
# train_df

cols = ["text","question","answer","answer_start","answer_end"]
comp_list = []
end_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        if row["data"]["answers"][i]["input_text"] == 'unknown':
          continue
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["span_start"])
        temp_list.append(row["data"]["answers"][i]["span_end"])
        comp_list.append(temp_list)
train_df = pd.DataFrame(comp_list, columns = cols) 
train_df

,text,question,answer,answer_start,answer_end
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475,151,179
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research,454,494
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law",457,511
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology",457,545
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project,769,879
...,...,...,...,...,...
107271,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was a sub?,Xabi Alonso,1405,1427
107272,(CNN) -- Cristiano Ronaldo provided the perfec...,Was it his first game this year?,Yes,1415,1467
107273,(CNN) -- Cristiano Ronaldo provided the perfec...,What position did the team reach?,third,1520,1555
107274,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was ahead of them?,Barca.,1557,1582


In [21]:
# fetching validation dataset from standford archive
coqa_dev = pd.read_json('https://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json')
coqa_dev

,version,data
0,1,"{'source': 'mctest', 'id': '3dr23u6we5exclen4t..."
1,1,"{'source': 'mctest', 'id': '3azhrg4cu4ktme1zh7..."
2,1,"{'source': 'race', 'id': '3ioen3p9s7jsqm9zwse0..."
3,1,"{'source': 'cnn', 'id': '36v4q8r5zk0iwte84nbw2..."
4,1,"{'source': 'mctest', 'id': '3a1pq49wvhh8nbtgsb..."
...,...,...
495,1,"{'source': 'race', 'id': '3npfyt4izc42dgjyfy8t..."
496,1,"{'source': 'mctest', 'id': '3a7y0r2p2ooc4i9zn4..."
497,1,"{'source': 'wikipedia', 'id': '3o7l7bfshep737y..."
498,1,"{'source': 'race', 'id': '3i02618ya06g9pi2dcnt..."


In [22]:
# deleting the unrequired columns
del coqa_dev["version"]
coqa_dev

,data
0,"{'source': 'mctest', 'id': '3dr23u6we5exclen4t..."
1,"{'source': 'mctest', 'id': '3azhrg4cu4ktme1zh7..."
2,"{'source': 'race', 'id': '3ioen3p9s7jsqm9zwse0..."
3,"{'source': 'cnn', 'id': '36v4q8r5zk0iwte84nbw2..."
4,"{'source': 'mctest', 'id': '3a1pq49wvhh8nbtgsb..."
...,...
495,"{'source': 'race', 'id': '3npfyt4izc42dgjyfy8t..."
496,"{'source': 'mctest', 'id': '3a7y0r2p2ooc4i9zn4..."
497,"{'source': 'wikipedia', 'id': '3o7l7bfshep737y..."
498,"{'source': 'race', 'id': '3i02618ya06g9pi2dcnt..."


In [24]:
# structuring the dev dataset just like training dataset
cols = ["text","question","answer","answer_start","answer_end"]
comp_list = []
end_list = []
for index, row in coqa_dev.iterrows():
    for i in range(len(row["data"]["questions"])):
        if row["data"]["answers"][i]["input_text"] == 'unknown':
          continue
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["span_start"])
        temp_list.append(row["data"]["answers"][i]["span_end"])
        comp_list.append(temp_list)
dev_df = pd.DataFrame(comp_list, columns = cols) 
dev_df

,text,question,answer,answer_start,answer_end
0,"Once upon a time, in a barn near a farm house,...",What color was Cotton?,white,59,93
1,"Once upon a time, in a barn near a farm house,...",Where did she live?,in a barn,18,80
2,"Once upon a time, in a barn near a farm house,...",Did she live alone?,no,196,215
3,"Once upon a time, in a barn near a farm house,...",Who did she live with?,with her mommy and 5 sisters,281,315
4,"Once upon a time, in a barn near a farm house,...",What color were her sisters?,orange and white,428,490
...,...,...,...,...,...
7912,"Las Vegas (, Spanish for ""The Meadows""), offic...","where does the nickname ""Sin City"" come from?",The city's tolerance for numerous forms of adu...,1037,1131
7913,"Las Vegas (, Spanish for ""The Meadows""), offic...",Which state is it in?,Nevada,100,207
7914,"Las Vegas (, Spanish for ""The Meadows""), offic...",Is it located in a desert?,Yes,326,358
7915,"Las Vegas (, Spanish for ""The Meadows""), offic...",what is the name of the desert?,Mojave Desert.,345,359


In [36]:
train_df.head()

,text,question,answer,answer_start,answer_end
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475,151,179
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research,454,494
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law",457,511
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology",457,545
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project,769,879


In [38]:
train_answer_df = train_df[['answer', 'answer_start', 'answer_end']].copy()
train_question_text_df = train_df[['text','question']].copy()

In [40]:
dev_answer_df = dev_df[['answer', 'answer_start', 'answer_end']].copy()
dev_question_text_df = dev_df[['text','question']].copy()

In [25]:
# train_contexts_coqa, train_questions_coqa, train_answers_coqa = list(train_df['text']), list(train_df['question']), list(train_df['answer'])
# valid_contexts_coqa, valid_questions_coqa, valid_answers_coqa = list(dev_df['text']), list(dev_df['question']), list(dev_df['answer'])

In [26]:
# print(f'There are {len(train_questions_coqa)} train questions')
# print(f'There are {len(valid_questions_coqa)} dev questions')

There are 107276 train questions
There are 7917 dev questions


In [27]:
# train_contexts_coqa[0], train_questions_coqa[0], train_answers_coqa[0:10]

('The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of 

In [34]:
type(train_questions_coqa)

list

In [46]:

# getting the model and its tokenizer (currently training on only 1000 rows as it is very time consuming)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings_coqa = tokenizer(train_question_text_df['text'][:1000].values.tolist(), train_question_text_df['question'][:1000].values.tolist(), truncation=True, padding=True)
valid_encodings_coqa = tokenizer(dev_question_text_df['text'][:100].values.tolist(), dev_question_text_df['question'][:100].values.tolist(), truncation=True, padding=True)

In [47]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [48]:
train_encodings_coqa.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [49]:
# printing the number of training data
no_of_encodings_coqa = len(train_encodings_coqa['input_ids'])
print(f'We have {no_of_encodings_coqa} context-question pairs')

We have 1000 context-question pairs


In [50]:
train_encodings_coqa[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [51]:
train_answer_df[:10]

,answer,answer_start,answer_end
0,It was formally established in 1475,151,179
1,research,454,494
2,"history, and law",457,511
3,"philosophy, science and theology",457,545
4,a project,769,879
5,into periods,1048,1127
6,five,1048,1128
7,The Vatican Apostolic Library,4,94
8,in Vatican City,94,150
9,1.1 million,328,412


In [53]:
# adding the answers in the training set for fine tuning
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    # print("--",answers[i]['answer_start'] if answers[i]['answer_start']!=-1 else 0)
    start_positions.append(encodings.char_to_token(i, answers.iloc[i]['answer_start'] ))
    # print("++", answers[i]['answer_end'] - 1)
    end_positions.append(encodings.char_to_token(i, answers.iloc[i]['answer_end'] -1 ))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings_coqa, train_answer_df[:1000])
add_token_positions(valid_encodings_coqa, dev_answer_df[:100])

In [54]:
class CoQA_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [55]:
train_dataset = CoQA_Dataset(train_encodings_coqa)
valid_dataset = CoQA_Dataset(valid_encodings_coqa)

In [56]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

In [61]:
# loading the BERT model which we will fine tune
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [62]:
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [63]:
# Fine tuning the model
N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)
model.to(device)
model.train()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [64]:
for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

Epoch 5: 100%|██████████| 125/125 [01:36<00:00,  1.30it/s, loss=0.879]


In [65]:
# checking the performance
model.eval()

acc = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)

100%|██████████| 13/13 [00:03<00:00,  4.12it/s]


In [66]:
acc

0.17307692307692307